# Model Analysis 

This program takes in datasets and models, and computes the loss landscapes over varying amounts of data.

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import time
import glob

# Importing our existing funcs
import os
import sys
from pathlib import Path

from minima_volume.dataset_funcs import ( load_models_and_data )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Model

In [2]:
# User specifies the model module name
from minima_volume.models import SVHN_model_data as model_module

# Generate dataset
#x_base, y_base, x_test, y_test = model_module.get_dataset(
#    device = device
#)

# Optional: you can keep hidden_dims or increase due to higher input size
hidden_dims = [512, 256]  # Suggested for SVHN, but your choice

# Grab model
model_template = model_module.get_model(hidden_dims=hidden_dims, device=device, seed=0)

# Grab loss and metrics
loss_fn_per_sample = model_module.get_loss_fn_per_sample()

## Loading Model and Datasets

In [3]:
# Get the relative path
target_dir = Path("models_and_data") #current directory

# Lists to store loaded models and additional data
loaded_models, loaded_additional_data, loaded_dataset = load_models_and_data(model_template=model_template, target_dir=target_dir, device="cpu")

# Print dataset information
print(f"Dataset type: {loaded_dataset['dataset_type']}")
print(f"Dataset quantities: {loaded_dataset['dataset_quantities']}")

# Print tensor shapes
print("\nTensor shapes:")
for key in ["x_base_train", "y_base_train", "x_additional", "y_additional", "x_test", "y_test"]:
    if loaded_dataset[key] is not None:
        print(f"  {key}: {loaded_dataset[key].shape}")
    else:
        print(f"  {key}: None")

Looking for models and dataset in: models_and_data
Found 6 model files:
  - model_additional_0.pt
  - model_additional_1940.pt
  - model_additional_19940.pt
  - model_additional_540.pt
  - model_additional_5940.pt
  - model_additional_59940.pt
✅ Model loaded into provided instance from models_and_data\model_additional_0.pt
Successfully loaded: model_additional_0.pt
✅ Model loaded into provided instance from models_and_data\model_additional_1940.pt
Successfully loaded: model_additional_1940.pt
✅ Model loaded into provided instance from models_and_data\model_additional_19940.pt
Successfully loaded: model_additional_19940.pt
✅ Model loaded into provided instance from models_and_data\model_additional_540.pt
Successfully loaded: model_additional_540.pt
✅ Model loaded into provided instance from models_and_data\model_additional_5940.pt
Successfully loaded: model_additional_5940.pt
✅ Model loaded into provided instance from models_and_data\model_additional_59940.pt
Successfully loaded: model_

✅ Dataset loaded from models_and_data\dataset.pt
Dataset type: data
Dataset quantities: [0, 540, 1940, 5940, 19940, 59940]

Tensor shapes:
  x_base_train: torch.Size([60, 3, 32, 32])
  y_base_train: torch.Size([60])
  x_additional: torch.Size([59940, 3, 32, 32])
  y_additional: torch.Size([59940])
  x_test: torch.Size([26032, 3, 32, 32])
  y_test: torch.Size([26032])


Base dataset size is  60

Extracted data:
Model 0: Dataset size = 60, Final test accuracy = 0.1517
Model 1: Dataset size = 600, Final test accuracy = 0.4195
Model 2: Dataset size = 2000, Final test accuracy = 0.5811
Model 3: Dataset size = 6000, Final test accuracy = 0.6979
Model 4: Dataset size = 20000, Final test accuracy = 0.7807
Model 5: Dataset size = 60000, Final test accuracy = 0.8326


Summary statistics:
Number of models: 6
Dataset sizes range: 60 to 60000
Accuracy range: 0.1517 to 0.8326
Mean accuracy: 0.5773

Saved data!


In [4]:
base_size = loaded_dataset['x_base_train'].shape[0]
print ("Base dataset size is ", base_size)

# Extract dataset sizes and final test accuracies
dataset_sizes = []
final_test_accuracies = []
for i, model_data in enumerate(loaded_additional_data):
    total_size = base_size + model_data['additional_data']
    final_test_accuracy = model_data['test_accs'][-1]
    dataset_sizes.append(total_size)
    final_test_accuracies.append(final_test_accuracy)

# Convert to numpy arrays for easier manipulation
dataset_sizes = np.array(dataset_sizes)
final_test_accuracies = np.array(final_test_accuracies)

sort_indices = np.argsort(dataset_sizes)
dataset_sizes = dataset_sizes[sort_indices]
final_test_accuracies = final_test_accuracies[sort_indices]

# Print the extracted data
print("\nExtracted data:")
for i, (size, acc) in enumerate(zip(dataset_sizes, final_test_accuracies)):
    print(f"Model {i}: Dataset size = {size}, Final test accuracy = {acc:.4f}")

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(dataset_sizes, final_test_accuracies, 'o-', markersize=8, linewidth=2)

# Customize the plot
plt.xlabel('Total Dataset Size', fontsize=12)
plt.ylabel('Final Test Accuracy', fontsize=12)
plt.title('Test Accuracy vs. Dataset Size', fontsize=14)
plt.grid(True, alpha=0.3)
plt.ylim((0.5, 1.0))
plt.tight_layout()
plt.show()

# Optional: Print summary statistics
print(f"\nSummary statistics:")
print(f"Number of models: {len(dataset_sizes)}")
print(f"Dataset sizes range: {dataset_sizes.min()} to {dataset_sizes.max()}")
print(f"Accuracy range: {final_test_accuracies.min():.4f} to {final_test_accuracies.max():.4f}")
print(f"Mean accuracy: {final_test_accuracies.mean():.4f}")

# Save the arrays to a file
np.savez("test_accuracies.npz", dataset_sizes=dataset_sizes, final_test_accuracies=final_test_accuracies)
print(f"\nSaved data!")